In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!ls /content/drive/MyDrive
#dataset_path = '/content/drive/MyDrive/newdataset/'
train_path='/content/newdataset/train'
test_path='/content/newdataset/test'
save1_path='/content/drive/MyDrive/my_model_dataset_BNNsmall_20k_200eb_163264285612_3classnew.h5'
save_path='/content/drive/MyDrive/DATASET2/BinaryNet_CNPB.h5'


In [2]:
!pip install -q kaggle

from google.colab import files
files.upload()
# Chọn file kaggle.json tải về ở bước 2 để tải lên

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kimthchnguyn/newdataset
!unzip -uq "/content/newdataset.zip"
#!kaggle datasets download -d kimthchnguyn/ccnp-newdataset
#!unzip -uq "/content/ccnp-newdataset.zip"


Saving kaggle.json to kaggle.json
 98% 482M/493M [00:05<00:00, 95.8MB/s]
100% 493M/493M [00:05<00:00, 96.5MB/s]


In [ ]:
#!pip install numpy==1.19.5
!pip uninstall tensorflow -y
!pip install tensorflow==2.7.1


In [ ]:
!pip install pyyaml h5py
!pip install larq

In [ ]:
import tensorflow as tf
from tensorflow import keras
import larq as lq
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths
from sklearn import preprocessing
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import cv2
from keras.utils import np_utils
print(tf.__version__)

In [7]:
img_width, img_height = 100, 100
numberOfLabels = 3
img_thresh =90
def convert_to_binary(img_grayscale, thresh=100):
    thresh, img_binary = cv2.threshold(img_grayscale, thresh, maxval=255, type=cv2.THRESH_BINARY)
    return img_binary

In [ ]:
data = []
labels = []

imagePaths = list(paths.list_images(train_path))
# loop over the image paths
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath, 0)
    image = convert_to_binary(image, thresh=img_thresh)
    image = cv2.resize(image, (img_width, img_height))
    image = cv2.bitwise_not(image)
    # image = image/255.0
    # fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
    #                 cells_per_block=(1, 1), visualize=True, multichannel=True)
    # print(image)
    data.append(image)
    labels.append(label)
#print(labels)
le = preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
#print (labels[1])
#print (image[1])
#print(labels)

data = np.array(data)
x_train = data.astype('float32')

x_train /= 255
y_train = np_utils.to_categorical(labels, numberOfLabels)
#######################################################################################################
data2 = []
labels2 = []

imagePaths = list(paths.list_images(test_path))
# loop over the image paths
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath, 0)
    image = convert_to_binary(image, thresh=img_thresh)
    image = cv2.resize(image, (img_width, img_height))
    image = cv2.bitwise_not(image)
    # image = image/255.0
    # fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
    #                 cells_per_block=(1, 1), visualize=True, multichannel=True)
    # print(image)
    data2.append(image)
    labels2.append(label)
#print(labels)
le = preprocessing.LabelEncoder()
le.fit(labels2)
labels = le.transform(labels2)
#print(labels)

data = np.array(data2)
x_test = data.astype('float32')

x_test /= 255
y_test = np_utils.to_categorical(labels2, numberOfLabels)

#print(y_train)
#img_width, img_height = 64, 64

#train_data_dir = 'data/train'
#validation_data_dir = 'data/test'
#nb_train_samples = 330
#nb_validation_samples = 60

print(x_train.shape )
print(x_test.shape )
print(y_train.shape )
print(y_test.shape )



CNN NEURAL NETWORK

In [9]:
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D,Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from keras.models import Sequential

model = Sequential()

# filters 32, size of filters (3,3)
model.add(Conv2D(8, (3, 3), input_shape=(img_width, img_height,1)))
model.add(BatchNormalization(axis=-1)) #BatchNormalization normalizes the matrix after it is been through a convolution layer so that the scale of each dimension remains the same
model.add(Activation('relu'))

model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) # act as an input to the Dense layers.

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(numberOfLabels))
model.add(BatchNormalization())
model.add(Activation('softmax')) # Softmax activation enables us to calculate the output based on the probabilities

BCNN NEURAL NETWORK

In [10]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip",
              use_bias=False)

model = tf.keras.models.Sequential([
    # In the first layer we only quantize the weights and not the input
    lq.layers.QuantConv2D(8, 3,
                          kernel_quantizer="ste_sign",
                          kernel_constraint="weight_clip",
                          use_bias=False,
                          input_shape=(img_width, img_height, 1)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(16, 3, padding="same", **kwargs),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(32, 3, padding="same", **kwargs),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(64, 3, padding="same", **kwargs),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(128, 3, padding="same", **kwargs),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),


    lq.layers.QuantConv2D(256, 3, padding="same", **kwargs),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    tf.keras.layers.Flatten(),

    lq.layers.QuantDense(512, **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantDense(numberOfLabels, **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Activation("softmax")
])

In [ ]:
lq.models.summary(model)

In [12]:
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
trained_model = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=100,
    validation_data=(x_test, y_test),
    shuffle=True
)

In [ ]:
#save mode
model.save(save_path)

In [ ]:
i=0
for layer in model.layers:
    print(layer.name, layer)
    print(model.layers[i].weights)
    i+=1
# print(model.layers[18].weights)
# model.save_weights('/content/drive/MyDrive/modelsave_weights.hdf5')

In [ ]:
plt.plot(trained_model.history['accuracy'])
#plt.plot(trained_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

print(np.max(trained_model.history['accuracy']))
print(np.max(trained_model.history['val_accuracy']))

In [ ]:
plt.plot(trained_model.history['loss'])
#plt.plot(trained_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

print(np.min(trained_model.history['loss']))
print(np.min(trained_model.history['val_loss']))